In [4]:
import os
import pandas as pd
import numpy as np
import re
from transformers import AutoTokenizer, AutoModel


c:\Users\liching\miniconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load tsv file and create a table for language translitration data.
with open(r"D:\creative idea\eng_mani\Dataset\mni_be_eng.tsv") as f:
    data = f.readlines()
split_data = list()
for lines in data:
    splitted_lines = lines.split('\t')
    split_data.append(splitted_lines)

curated_data = pd.DataFrame(split_data, columns= ['eng', 'mani'])
curated_data['eng'] = curated_data['eng'].str.lower()

curated_data.head()

,eng,mani
0,the union cabinet chaired by prime minister sh...,প্রধানমন্ত্রী শ্রী নরেন্দ্র মোদীনা লুচিংবা কেন...
1,"the time frame for implementation is one year,",ইমপ্লিমেন্ত তৌবগী তাইম ফ্রেম অসি চহী অমনি ।\n
2,rashtriya arogya nidhi (ran) was set up as a r...,রাষ্ট্রিয়া অরোগ্যা নিধী (আরএএন) অসি অকক্নবা কে...
3,since the dohfw provides funds to the hospital...,দিওঐছএফদবল্যুনা হোস্পিতালশিংদা ফন্দশিং পিথবা ও...
4,there has been no continuous funding to jsk fr...,মন্ত্রালয়দগী জেএসকেদা মখা তাবা শেন্থং ফদ্রে ।\n


In [9]:
# load model and tokenizer.

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModel.from_pretrained("google/flan-t5-base")


In [19]:
# tokenise the data 
max_length = 512

def preprocess_function(examples):
  inputs = [examples["eng"].iloc[ex] for ex in examples.index]
  targets = [examples["mani"].iloc[ex] for ex in examples.index]
 
  model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
  labels = tokenizer(targets,max_length=max_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
 
  return model_inputs

mi = preprocess_function(curated_data)


In [56]:
inp = tokenizer(curated_data['eng'].iloc[0], max_length=512, truncation=True, return_tensors= 'pt')
print(inp)

{'input_ids': tensor([[    8,  7021,  4566,  3533,    15,    26,    57,  3427,  6323,     3,
         31763,     3,    29,   291, 12524,  1794,    23,    65,  3754,     8,
          6384,    21, 12493,    13, 21286,  5678,     6,     3, 17332,     6,
             3, 12380,    17, 26038,     3,     9,  3822,    63,     9,     3,
            29,    23,    26,   107,    23,    41,  2002,    61,    11,     3,
          7066,     7,  5979,   107,    63,     9,     3,     7,  7436,  9300,
             3,  9692,   107,    41,   354,     7,   157,    61,    11,     8,
          3621,    33,  4382,    12,    36,     3,   162,  6265,    16,  3066,
            13,   533,     3,   184,   384, 13454,    41,    26,    32,   107,
            89,   210,   137,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [57]:
inp_lab = tokenizer(curated_data['mani'].iloc[0], max_length=512, truncation=True, return_tensors= 'pt')
print(inp)

{'input_ids': tensor([[    8,  7021,  4566,  3533,    15,    26,    57,  3427,  6323,     3,
         31763,     3,    29,   291, 12524,  1794,    23,    65,  3754,     8,
          6384,    21, 12493,    13, 21286,  5678,     6,     3, 17332,     6,
             3, 12380,    17, 26038,     3,     9,  3822,    63,     9,     3,
            29,    23,    26,   107,    23,    41,  2002,    61,    11,     3,
          7066,     7,  5979,   107,    63,     9,     3,     7,  7436,  9300,
             3,  9692,   107,    41,   354,     7,   157,    61,    11,     8,
          3621,    33,  4382,    12,    36,     3,   162,  6265,    16,  3066,
            13,   533,     3,   184,   384, 13454,    41,    26,    32,   107,
            89,   210,   137,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [58]:
inp['labels'] = inp_lab['input_ids']
inp

{'input_ids': tensor([[    8,  7021,  4566,  3533,    15,    26,    57,  3427,  6323,     3,
         31763,     3,    29,   291, 12524,  1794,    23,    65,  3754,     8,
          6384,    21, 12493,    13, 21286,  5678,     6,     3, 17332,     6,
             3, 12380,    17, 26038,     3,     9,  3822,    63,     9,     3,
            29,    23,    26,   107,    23,    41,  2002,    61,    11,     3,
          7066,     7,  5979,   107,    63,     9,     3,     7,  7436,  9300,
             3,  9692,   107,    41,   354,     7,   157,    61,    11,     8,
          3621,    33,  4382,    12,    36,     3,   162,  6265,    16,  3066,
            13,   533,     3,   184,   384, 13454,    41,    26,    32,   107,
            89,   210,   137,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [50]:
inp['input_ids']

tensor([[    8,  7021,  4566,  3533,    15,    26,    57,  3427,  6323,     3,
         31763,     3,    29,   291, 12524,  1794,    23,    65,  3754,     8,
          6384,    21, 12493,    13, 21286,  5678,     6,     3, 17332,     6,
             3, 12380,    17, 26038,     3,     9,  3822,    63,     9,     3,
            29,    23,    26,   107,    23,    41,  2002,    61,    11,     3,
          7066,     7,  5979,   107,    63,     9,     3,     7,  7436,  9300,
             3,  9692,   107,    41,   354,     7,   157,    61,    11,     8,
          3621,    33,  4382,    12,    36,     3,   162,  6265,    16,  3066,
            13,   533,     3,   184,   384, 13454,    41,    26,    32,   107,
            89,   210,   137,     1]])

In [59]:
de_en = tokenizer.convert_ids_to_tokens(inp['input_ids'][0])
print(f"english: {de_en}")

de_man = tokenizer.convert_ids_to_tokens(inp['labels'][0])

print(f"mani: {de_man}")

english: ['▁the', '▁union', '▁cabinet', '▁chair', 'e', 'd', '▁by', '▁prime', '▁minister', '▁', 'shri', '▁', 'n', 'ar', 'endra', '▁mod', 'i', '▁has', '▁approved', '▁the', '▁proposal', '▁for', '▁closure', '▁of', '▁autonomous', '▁bodies', ',', '▁', 'namely', ',', '▁', 'rash', 't', 'riya', '▁', 'a', 'rog', 'y', 'a', '▁', 'n', 'i', 'd', 'h', 'i', '▁(', 'ran', ')', '▁and', '▁', 'jan', 's', 'ank', 'h', 'y', 'a', '▁', 's', 'thi', 'rata', '▁', 'kos', 'h', '▁(', 'j', 's', 'k', ')', '▁and', '▁the', '▁functions', '▁are', '▁proposed', '▁to', '▁be', '▁', 've', 'sted', '▁in', '▁department', '▁of', '▁health', '▁', '&', '▁family', '▁welfare', '▁(', 'd', 'o', 'h', 'f', 'w', ').', '</s>']
mani: ['▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁(', '<unk>', ')', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁(', '<unk>', ')', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>', '▁', '<unk>',

In [60]:
de_man = tokenizer.decode(inp['labels'][0])

print(f"mani: {de_man}")

mani: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> (<unk>) <unk> <unk> <unk> <unk> (<unk>)<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> (<unk>)<unk> <unk> <unk> <unk> <unk> <unk> <unk> </s>


In [61]:
de_en = tokenizer.decode(inp['input_ids'][0])
print(f"english: {de_en}")

english: the union cabinet chaired by prime minister shri narendra modi has approved the proposal for closure of autonomous bodies, namely, rashtriya arogya nidhi (ran) and jansankhya sthirata kosh (jsk) and the functions are proposed to be vested in department of health & family welfare (dohfw).</s>


# Meitei mayek

In [20]:
eng_path = r"D:\creative idea\eng_mani\Dataset\data_mni-Mtei\en-mni-Mtei\dev\txt\dev.en"
mani_path = r"D:\creative idea\eng_mani\Dataset\data_mni-Mtei\en-mni-Mtei\dev\txt\dev.mni-Mtei.txt"

with open(eng_path) as eng_f:
    eng_data = eng_f.read()
with open(mani_path, encoding="utf-8") as mani_f:
    mani_data = mani_f.read()

In [21]:
combined_lang = [[english_line, manipuri_line] for english_line, manipuri_line in zip(eng_data.split("\n"), mani_data.split("\n"))]
new_curated_data = pd.DataFrame(combined_lang, columns= ['eng', 'mani'])
new_curated_data.eng =new_curated_data.eng.str.lower()
new_curated_data.head()

,eng,mani
0,in chapter 3 arjun said lord,ꯑꯙ꯭ꯌꯥꯌ ꯳ꯗꯥ ꯑꯔꯖꯨꯅꯅꯥ ꯍꯥꯌꯈꯤ ꯚꯒꯕꯥꯟ꯫
1,why do you cast me into dreadful acts when you...,ꯂꯧꯁꯤꯡꯒꯤ ꯂꯝꯕꯤ ꯑꯁꯤ ꯍꯦꯟꯅꯥ ꯐꯩ ꯍꯥꯌꯅꯥ ꯊꯥꯖꯔꯀꯄꯥ ꯃꯇꯃꯗꯥ ...
2,he found the way of knowledge easier to practi...,ꯃꯍꯥꯛꯅꯥ ꯂꯧꯁꯤꯡꯒꯤ ꯂꯝꯕꯤ ꯑꯁꯤ ꯆꯠꯅꯍꯅꯕꯗꯥ ꯍꯦꯟꯅꯥ ꯂꯥꯌꯕꯥ ꯎ...
3,so there is profit in both success and failure,ꯃꯔꯝ ꯑꯗꯨꯅꯥ ꯃꯥꯌ ꯄꯥꯀꯄꯥ ꯑꯃꯁꯨꯡ ꯃꯥꯌ ꯄꯥꯀꯄꯥ ꯑꯅꯤꯃꯛꯇꯥ ꯀꯥ...
4,but by now he understands well that action a p...,ꯑꯗꯨꯕꯨ ꯍꯧꯖꯤꯛ ꯐꯥꯑꯣꯕꯗꯥ ꯃꯍꯥꯛꯅꯥ ꯐꯖꯅꯥ ꯈꯉꯂꯦ ꯃꯗꯨꯗꯤ ꯑꯦꯛ...


In [22]:
# save this data for training as csv
# from pathlib import Path

# save_path = Path("../Dataset/meitei_eng/train.csv")
# new_curated_data.to_csv(save_path)

In [5]:
# load model and tokenizer.

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModel.from_pretrained("google/flan-t5-base")

In [23]:
# train a tokemizer
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# Paths to your Meitei text files
files = [str(x) for x in new_curated_data.mani.tolist()]

# Initialize a Byte-Level BPE tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training (adjust vocab size, min frequency, and special tokens)
tokenizer.train_from_iterator(iterator=files, vocab_size=52_000, min_frequency=2, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

# # Save the tokenizer files to disk
# # run one time
# tokenizer.save_model(".", "meitei_tokenizer")


In [28]:
# testing
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "../meitei_tokenizer/meitei_tokenizer-vocab.json",
    "../meitei_tokenizer/meitei_tokenizer-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")), # sepearation token
    ("<s>", tokenizer.token_to_id("<s>")), # cls token
)
tokenizer.enable_truncation(max_length=512)

manipuri_text = "ꯑꯙ꯭ꯌꯥꯌ ꯳ꯗꯥ ꯑꯔꯖꯨꯅꯅꯥ ꯍꯥꯌꯈꯤ ꯚꯒꯕꯥꯟ꯫"

encoded_manipuri = tokenizer.encode(manipuri_text, add_special_tokens=True)
print(encoded_manipuri)
print(encoded_manipuri.tokens)

print(tokenizer.decode(encoded_manipuri.ids))


Encoding(num_tokens=24, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['<s>', 'ê¯ĳ', 'ê¯Ļ', 'ê¯Ń', 'ê¯Į', 'ê¯¥', 'ê¯Į', 'Ġê¯', '³', 'ê¯Ĺ', 'ê¯¥', 'Ġê¯ĳê¯Ķê¯ĸ', 'ê¯¨', 'ê¯ħê¯ħ', 'ê¯¥', 'Ġê¯į', 'ê¯¥', 'ê¯Įê¯Ī', 'ê¯¤', 'Ġê¯ļê¯Ĵê¯ķ', 'ê¯¥', 'ê¯Ł', 'ê¯«', '</s>']
<s>ꯑꯙ꯭ꯌꯥꯌ ꯳ꯗꯥ ꯑꯔꯖꯨꯅꯅꯥ ꯍꯥꯌꯈꯤ ꯚꯒꯕꯥꯟ꯫</s>


In [24]:
print(encoded_manipuri.ids)

[0, 307, 791, 290, 276, 263, 276, 262, 116, 271, 263, 669, 266, 475, 263, 306, 263, 396, 264, 720, 263, 298, 299, 2]


# working

In [29]:
# import required library
import json
import os
import logging
import pandas as pd

from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, AutoModel

# max_length
max_length = 512

# laod custom manipuri tokenizer
def mn_tokenizer(manipuri_text):
    # Tokenize the Manipuri text using the Byte-level BPE Tokenizer and the BertProcessing processor.
    tokenizer = ByteLevelBPETokenizer(
        "../meitei_tokenizer/meitei_tokenizer-vocab.json",
        "../meitei_tokenizer/meitei_tokenizer-merges.txt",
                                    )
    
    tokenizer._tokenizer.post_processor = BertProcessing(
        ("</s>", tokenizer.token_to_id("</s>")), # sepearation token
        ("<s>", tokenizer.token_to_id("<s>")), # cls token
    )
    tokenizer.enable_truncation(max_length=max_length) # max length of the sequence.

    return (tokenizer.encode(manipuri_text, add_special_tokens=True))

# tokenizer decoder
def mn_decoder(manipuri_text_ids):
    # Tokenize the Manipuri text using the Byte-level BPE Tokenizer and the BertProcessing processor.
    tokenizer = ByteLevelBPETokenizer(
        "../meitei_tokenizer/meitei_tokenizer-vocab.json",
        "../meitei_tokenizer/meitei_tokenizer-merges.txt",
                                    )
    return tokenizer.decode(manipuri_text_ids)

# load english tokenizer and model
eng_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModel.from_pretrained("google/flan-t5-base")

# load train dataset
data = pd.read_csv("../Dataset/meitei_eng/train.csv")

In [30]:
# split data into train val
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=False)

In [31]:
val_data.reset_index(inplace= True)
val_data.index

RangeIndex(start=0, stop=354, step=1)

In [32]:
train_data.index

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414],
      dtype='int64', length=1415)

In [43]:
# curate a token dataset.
def preprocess_function(dataset, en_tokenizer):
  inputs = [dataset["eng"].iloc[ex] for ex in dataset.index]
  targets = [dataset["mani"].iloc[ex] for ex in dataset.index]
 
  model_inputs = en_tokenizer(inputs, max_length=max_length, truncation=True)
  labels = list()
  for target in targets:
    # print(target)
    label = mn_tokenizer(target)
    ids = label.ids
    labels.append(ids)
    
  model_inputs["labels"] = labels
 
  return model_inputs

train_set = preprocess_function(train_data, eng_tokenizer)
val_set = preprocess_function(val_data, eng_tokenizer)
# print(mi)

mi = preprocess_function(data, eng_tokenizer)

AttributeError: 'NoneType' object has no attribute 'pad'

In [34]:
print(mi['labels'])

[[0, 307, 791, 290, 276, 263, 276, 262, 116, 271, 263, 669, 266, 475, 263, 306, 263, 396, 264, 720, 263, 298, 299, 2], [0, 300, 302, 278, 264, 341, 264, 436, 264, 329, 264, 306, 283, 346, 263, 330, 288, 306, 263, 297, 263, 296, 263, 1495, 263, 385, 263, 369, 264, 277, 264, 269, 288, 267, 266, 481, 264, 267, 263, 294, 270, 581, 263, 569, 264, 315, 263, 296, 263, 271, 270, 308, 264, 274, 264, 428, 749, 2], [0, 418, 263, 326, 263, 281, 302, 278, 264, 341, 264, 436, 264, 329, 264, 1651, 263, 306, 283, 346, 263, 281, 263, 351, 263, 767, 264, 422, 264, 345, 264, 277, 264, 488, 270, 272, 329, 264, 268, 263, 303, 263, 443, 264, 267, 263, 294, 264, 397, 263, 385, 263, 313, 290, 1548, 264, 294, 266, 448, 264, 292, 266, 272, 303, 263, 441, 263, 594, 264, 386, 270, 370, 263, 539, 296, 270, 308, 263, 330, 264, 625, 295, 266, 296, 263, 381, 263, 294, 264, 274, 323, 2], [0, 955, 295, 266, 268, 263, 303, 263, 276, 294, 263, 308, 263, 292, 266, 272, 303, 263, 276, 294, 263, 308, 263, 456, 264, 917, 263

In [35]:
print(mn_decoder(mi['labels'][1]))

<s>ꯂꯧꯁꯤꯡꯒꯤ ꯂꯝꯕꯤ ꯑꯁꯤ ꯍꯦꯟꯅꯥ ꯐꯩ ꯍꯥꯌꯅꯥ ꯊꯥꯖꯔꯀꯄꯥ ꯃꯇꯃꯗꯥ ꯀꯔꯤꯒꯤ ꯑꯩꯕꯨ ꯑꯀꯤꯕꯥ ꯄꯣꯀꯍꯅꯕꯥ ꯊꯕꯀꯁꯤꯡꯗꯥ ꯊꯥꯗꯣꯀꯄꯤꯔꯤꯕꯅꯣ꯫</s>


In [36]:
train_set['input_ids']

[[16, 5800, 220, 1584, 6959, 243, 3, 322, 26, 1],
 [572,
  103,
  25,
  4061,
  140,
  139,
  3,
  26,
  5236,
  1329,
  6775,
  116,
  25,
  857,
  24,
  8,
  194,
  13,
  1103,
  19,
  4784,
  1],
 [3,
  88,
  435,
  8,
  194,
  13,
  1103,
  1842,
  12,
  1032,
  21,
  48,
  194,
  7992,
  7,
  3,
  28202,
  280,
  16,
  8,
  605,
  13,
  9589,
  11,
  8,
  538,
  13,
  5737,
  12582,
  16,
  24,
  13,
  6224,
  1],
 [78, 132, 19, 3199, 16, 321, 1269, 11, 3338, 1],
 [68, 57, 230, 3, 88, 734, 7, 168, 24, 1041, 3, 9, 26840, 13, 321, 8, 1155, 1],
 [3,
  10648,
  7,
  107,
  29,
  65,
  92,
  3,
  19492,
  376,
  12,
  5413,
  2448,
  13,
  3228,
  7,
  11,
  240,
  14330,
  16,
  142,
  49,
  250,
  19,
  8,
  163,
  1391,
  13,
  8,
  4349,
  3,
  88,
  2762,
  7,
  1],
 [983,
  274,
  3,
  88,
  854,
  7,
  80,
  13,
  8,
  192,
  1155,
  1584,
  6959,
  656,
  3,
  9,
  15084,
  8121,
  1],
 [1584,
  6959,
  243,
  25,
  43,
  78,
  623,
  3,
  30868,
  3,
  32,
  3,
  10648,
  7,
 

# model training 

In [37]:
print(model)

T5Model(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo): Linear(in_features

In [38]:
# training the model

In [39]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }
        
train_X = CustomDataset(input_ids= train_set['input_ids'], attention_mask= train_set['attention_mask'], labels= train_set['labels'])
val_X = CustomDataset(input_ids= val_set['input_ids'], attention_mask= val_set['attention_mask'], labels= val_set['labels'])

In [40]:
# Set up training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="../meitei_eng/output", # output directory for saving model checkpoints and logs.
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    evaluation_strategy="epoch", # evaluate after every epoch.
    save_total_limit=2,
    eval_steps=500,
    logging_steps=500,
    learning_rate=5e-5,
    save_steps=500,
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_X,
    eval_dataset=val_X
)

trainer.train()

  0%|          | 0/1770 [00:00<?, ?it/s]

ValueError: expected sequence of length 18 at dim 1 (got 35)

In [42]:
train_X[1]

{'input_ids': [572,
  103,
  25,
  4061,
  140,
  139,
  3,
  26,
  5236,
  1329,
  6775,
  116,
  25,
  857,
  24,
  8,
  194,
  13,
  1103,
  19,
  4784,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [0,
  300,
  302,
  278,
  264,
  341,
  264,
  436,
  264,
  329,
  264,
  306,
  283,
  346,
  263,
  330,
  288,
  306,
  263,
  297,
  263,
  296,
  263,
  1495,
  263,
  385,
  263,
  369,
  264,
  277,
  264,
  269,
  288,
  267,
  266,
  481,
  264,
  267,
  263,
  294,
  270,
  581,
  263,
  569,
  264,
  315,
  263,
  296,
  263,
  271,
  270,
  308,
  264,
  274,
  264,
  428,
  749,
  2]}

# sjhdjsadjk

In [51]:
# import required library
import json
import os
import logging
import pandas as pd
from sklearn.model_selection import train_test_split

from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, AutoModel

# max_length
max_length = 512

# laod custom manipuri tokenizer
def mn_tokenizer(manipuri_text):
    # Tokenize the Manipuri text using the Byte-level BPE Tokenizer and the BertProcessing processor.
    tokenizer = ByteLevelBPETokenizer(
        "../meitei_tokenizer/meitei_tokenizer-vocab.json",
        "../meitei_tokenizer/meitei_tokenizer-merges.txt",
                                    )
    
    tokenizer._tokenizer.post_processor = BertProcessing(
        ("</s>", tokenizer.token_to_id("</s>")), # sepearation token
        ("<s>", tokenizer.token_to_id("<s>")), # cls token
    )
    tokenizer.enable_truncation(max_length=max_length) # max length of the sequence.

    return (tokenizer.encode(manipuri_text, add_special_tokens=True))

# tokenizer decoder
def mn_decoder(manipuri_text_ids):
    # Tokenize the Manipuri text using the Byte-level BPE Tokenizer and the BertProcessing processor.
    tokenizer = ByteLevelBPETokenizer(
        "../meitei_tokenizer/meitei_tokenizer-vocab.json",
        "../meitei_tokenizer/meitei_tokenizer-merges.txt",
                                    )
    return tokenizer.decode(manipuri_text_ids)

# load english tokenizer and model
eng_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModel.from_pretrained("google/flan-t5-base")

# load train dataset
data = pd.read_csv("../Dataset/meitei_eng/train.csv")

# mi = preprocess_function(data, eng_tokenizer)
# print(mi)
# split data into train val
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=False)
val_data.reset_index(inplace= True) # reset index for customdata loader


In [53]:
# curate a token dataset.
def preprocess_function(dataset, en_tokenizer):
    inputs = [dataset["eng"].iloc[ex] for ex in dataset.index]
    targets = [dataset["mani"].iloc[ex] for ex in dataset.index]
 
    model_inputs = en_tokenizer(inputs, max_length=max_length, truncation=True)
    labels = list()
    for target in targets:
        # print(target)
        label = mn_tokenizer(target)
        ids = label.ids
        labels.append(ids)

    print("Length of input sequences:", len(model_inputs["input_ids"]))
    print("Length of labels:", len(labels))  
    model_inputs["labels"] = labels
    
    return model_inputs
#
train_set = preprocess_function(train_data, eng_tokenizer)
val_set = preprocess_function(val_data, eng_tokenizer)

# Model training
# loading dataset using custom data loader
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }
        
train_X = CustomDataset(input_ids= train_set['input_ids'], attention_mask= train_set['attention_mask'], labels= train_set['labels'])
val_X = CustomDataset(input_ids= val_set['input_ids'], attention_mask= val_set['attention_mask'], labels= val_set['labels'])

# Set up training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./meitei_eng/output", # output directory for saving model checkpoints and logs.
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    evaluation_strategy="epoch", # evaluate after every epoch.
    save_total_limit=2,
    eval_steps=500,
    logging_steps=500,
    learning_rate=5e-5,
    save_steps=500,
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_X,
    eval_dataset=val_X
)

trainer.train()

Length of input sequences: 1415
Length of labels: 1415
Length of input sequences: 354
Length of labels: 354


  0%|          | 0/1770 [00:00<?, ?it/s]

ValueError: expected sequence of length 18 at dim 1 (got 35)